## Week 9 Homework

1. Download the UCI Heart Disease dataset on Kaggle:  
    https://www.kaggle.com/ronitf/heart-disease-uci

In [1]:
from mlens.ensemble import SuperLearner
from mlens.model_selection import Evaluator
from mlens.metrics import make_scorer
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.svm import SVC
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

[MLENS] backend: threading


In [2]:
df = pd.read_csv("heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [4]:
X = df.drop(columns = 'target')
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=555, stratify=y)

2. Calculate the accuracy of each of the following at estimating the presence of heart disease
    * Logistic Regression
    * Random Forest
    * Support Vector Machine
    * KNeighbors
    * Decision Tree
    * Ada Boost
    * Bagging
    * Gradient Boosting
    * XGBoost

In [7]:
LR = LogisticRegression(solver='lbfgs', max_iter=10000, random_state=555)
RF = RandomForestClassifier(n_estimators = 100, random_state=555)
SVM = SVC(random_state=0, probability=True)
KNC = KNeighborsClassifier()
DTC = DecisionTreeClassifier()
ABC = AdaBoostClassifier(n_estimators = 100)
BC = BaggingClassifier(n_estimators = 100)
GBC = GradientBoostingClassifier(n_estimators = 100)
# clf_XGB = XGBClassifier(n_estimators = 100, objective= 'binary:logistic', seed=555, use_label_encoder=False)
clf_XGB = XGBClassifier(n_estimators = 100, seed=555, use_label_encoder=False, eval_metric='logloss')
clfs = []
print('5-fold cross validation:\n')
for clf, label in zip([LR, RF, KNC, DTC, ABC, BC, GBC, clf_XGB],
                      ['Logistic Regression',
                       'Random Forest',
                       #'Support Vector Machine',
                       'KNeighbors',
                       'Decision Tree',
                       'Ada Boost',
                       'Bagging',
                       'Gradient Boosting',
                       'XGBoost']):
    scores = sklearn.model_selection.cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
    print("Train CV Accuracy: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(X_train, y_train)
    clfs.append(md)
    print("Test Accuracy: %0.4f " % (sklearn.metrics.accuracy_score(clf.predict(X_test), y_test)))

5-fold cross validation:

Train CV Accuracy: 0.841 (+/- 0.039) [Logistic Regression]
Test Accuracy: 0.7763 
Train CV Accuracy: 0.846 (+/- 0.055) [Random Forest]
Test Accuracy: 0.8026 
Train CV Accuracy: 0.687 (+/- 0.049) [KNeighbors]
Test Accuracy: 0.6053 
Train CV Accuracy: 0.793 (+/- 0.037) [Decision Tree]
Test Accuracy: 0.7500 
Train CV Accuracy: 0.762 (+/- 0.043) [Ada Boost]
Test Accuracy: 0.7763 
Train CV Accuracy: 0.789 (+/- 0.036) [Bagging]
Test Accuracy: 0.7895 
Train CV Accuracy: 0.819 (+/- 0.026) [Gradient Boosting]
Test Accuracy: 0.8026 
Train CV Accuracy: 0.828 (+/- 0.030) [XGBoost]
Test Accuracy: 0.7895 


3. Which model performed best in #2 ?

In [8]:
'Random Forest and Gradient Boosting have the highest test accuracy of 0.8026. They are the best models.'

'Random Forest and Gradient Boosting have the highest test accuracy of 0.8026. They are the best models.'

4. Find a preprocessing technique we have used in class to improve your best model from above.

# In my case 4 and 5 are identical because I have 2 best models, not just one. I do it in 5.

In [13]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [14]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

5. Use the ML-Ensemble library to combine your best model and your second best model into a single layer.  
    http://ml-ensemble.com/

In [16]:
ensemble = SuperLearner(scorer=accuracy_score, random_state=555, verbose=2)

In [18]:
# Build the first layer
ensemble.add([RF, GBC])

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...B909EE0>)],
   n_jobs=-1, name='group-0', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x00000227EB909EE0>,
       shuffle=False, verbose=2)

In [19]:
# Attach the final meta estimator
ensemble.add_meta(LogisticRegression())

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...B909EE0>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x00000227EB909EE0>,
       shuffle=False, verbose=2)

In [20]:
# Fit ensemble
ensemble.fit(X_train, y_train)


Fitting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:01


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...B909EE0>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x00000227EB909EE0>,
       shuffle=False, verbose=2)

In [21]:
#pred_vals = ensemble.predict(X_test)
print ("Accuracy - Train : ", sklearn.metrics.accuracy_score(ensemble.predict(X_train), y_train))
print ("Accuracy - Test : ", sklearn.metrics.accuracy_score(ensemble.predict(X_test), y_test))


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Accuracy - Train :  1.0

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Accuracy - Test :  0.8026315789473685


In [22]:
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                       score-m  score-s  ft-m  ft-s  pt-m  pt-s
layer-1  gradientboostingclassifier       0.78     0.00  0.77  0.04  0.00  0.00
layer-1  randomforestclassifier           0.80     0.02  0.48  0.01  0.02  0.01



6. Create a custom model using any technique you prefer.  The (two) students who find the best two models in the class will receive +2% on their final exam score.  Any python ML model using techniques we have studied in class are acceptable.